# Демонстрация отбора признаков с помощью метода Target Permutation

## Структура ноутбука:
- Демонстрация методов и атрибутов библиотеки Target Permutation для задач классификации
- Применение библиотеки Target Permutation для задач классификации
    * Проверка Target Permutation на искусственных данных
    * Проверка Target Permutation на данных (Титаник)
    
    
- Применение библиотеки Target Permutation для задач регрессии
    * Проверка Target Permutation на искусственных данных
    * Проверка Target Permutation на данных (Стоимость жилья)    

##  Подгрузим необходимые бибиотеки

In [1]:
# расширяем поле, убираем предупреждения
from IPython.core.display import display, HTML
display(HTML('<style>.container {width:95% !important;}</style>'))

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Стандартные библиотеки для анализа
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, mean_absolute_percentage_error

import matplotlib.pylab as pl

In [3]:
import sys
sys.path.append('../')

# Наша разработанная библиотека
from autobinary import TargetPermutationSelection, base_pipe #, utils_target_permutation
# подгрузим вспомогательные функции
from autobinary import utils_target_permutation
# from utils_target_permutation import select_estimator, geom_reg_list, aggregated_feature_selection, intersection_set

In [4]:
import sklearn as sk
import xgboost as xgb
import matplotlib as mplot
import autobinary as ab

print('sklearn: ', sk.__version__)
print('xgboost: ', xgb.__version__)
print('matplotlib: ', mplot.__version__)
print('autobinary: ', ab.__version__)

sklearn:  1.1.3
xgboost:  1.5.2
matplotlib:  3.7.0
autobinary:  1.0.9


## Генерация искусственных данных

In [5]:
from sklearn.datasets import make_classification

X,y = make_classification(
    n_samples=10000,
    n_features=10,
    n_informative=5,
    n_classes=2,
    n_clusters_per_class=2,
    class_sep=2,
    weights=[0.5, 0.5],
    random_state=2022)

columns = ['A','B','C','D','E','F','G','H','I','J']
sample = pd.DataFrame(X, columns=columns)
sample['target'] = y
target = 'target'

X_train, X_valid, y_train, y_valid = train_test_split(
    sample.drop(columns=target), 
    sample[target], 
    test_size=0.3, 
    random_state=2022)

In [6]:
num_columns = columns
cat_columns = []
target = ['target']

#pipeline
prep_pipe = base_pipe(
    num_columns=num_columns,
    cat_columns=cat_columns)

Определены только количественные переменные!


## Демострация методов и атрибутов Target Permutation

Описание: Класс **TargetPermutationSelection** предназначен для отбора признаков с помощью перемешивания таргета и оценки предсказательной способности фичей.

Последовательность действий алгоритма:
- `1)` Происходит обучение модели с оригинальным тарегетом (benchmark-модель);
- `2)` Происходит расчет метрики benchmark-модели;
- `3)` Происходит перемешивание таргета;
- `4)` Происходит обучение модели с перемешаным таргетом (target permutation-модель);
- `5)` Происходит расчет метрики target permutation-модели;
- `6)` Расчет разности метрик 2) и 5)
- `7)` Происходит отбор признаков

In [7]:
from xgboost import XGBClassifier

params_xgb = {
    'eta':0.01,
    'n_estimators':800,
    'subsample':0.9,
    'colsample_bytree':0.6,
    'max_depth':6,
    'objective':'binary:logistic',
    'use_label_encoder':False,
    'n_jobs':-1,
    'random_state':42}

fit_params_xgb = {
    'early_stopping_rounds':100, 
    'eval_metric':['logloss', 'aucpr', 'auc'],
    'verbose':0}

estimator = XGBClassifier(**params_xgb)

In [8]:
# инициализация метода
model = TargetPermutationSelection(
    main_estimator=estimator, 
    main_fit_params=fit_params_xgb,
    main_prep_pipe=prep_pipe,
    main_features=num_columns + cat_columns,

    X_train=sample[num_columns + cat_columns],
    y_train=sample[target], 
    main_metric='gini',
    model_type='xgboost')

# метод для вычисления важностей фичей
model.calculate_permutation(test_size=0.3)

============================== 1. Обучение benchmark-модели! ==============================
BEST ITERATION:  61
gini на обучающей выборке: 99.433
gini на проверочной выборке: 99.001
============================== 2. Обучение target permutation-модели! ==============================
BEST ITERATION:  56
gini на обучающей выборке: 53.888
gini на проверочной выборке: 1.161


,index,importance_benchmark,importance_permut,final_importance
0,F,230.534286,4.124747,226.409539
1,A,65.284569,4.149655,61.134913
2,J,63.469402,4.574215,58.895187
3,C,32.615238,4.260645,28.354593
4,G,29.989086,4.402431,25.586655
5,B,20.135345,4.357924,15.777422
6,D,8.951872,4.125734,4.826138
8,E,1.624735,4.506929,-2.882194
7,H,1.691387,4.580022,-2.888635
10,I,1.200047,4.200210,-3.000163


In [9]:
# отбор фичей с помощью пересечения множеств
model.select_features(select_type='random_feature', cut_off=0.0) # для отбора фичей, которые результативнее рандомной фичи
# model.select_features(select_type='any', cut_off=0.0) # для отбора фичей, важность которых больше cut_off

10 признаков было до Target Permutation
10 признаков после Target Permutation


['F', 'A', 'J', 'C', 'G', 'B', 'D', 'E', 'H', 'I']

In [10]:
# атрибуты
model.selected_features # отобранные фичи

['F', 'A', 'J', 'C', 'G', 'B', 'D', 'E', 'H', 'I']

In [11]:
model.df_benchmark_fi # датафрейм с важностями для benchmark-модели

,index,importance_benchmark
0,F,230.534286
1,A,65.284569
2,J,63.469402
3,C,32.615238
4,G,29.989086
5,B,20.135345
6,D,8.951872
7,H,1.691387
8,E,1.624735
9,random_feature,1.452201


In [12]:
model.df_permutation_fi # датафрейм с важностями для target permutation-модели

,index,importance_permut
0,random_feature,4.581307
1,H,4.580022
2,J,4.574215
3,E,4.506929
4,G,4.402431
5,B,4.357924
6,C,4.260645
7,I,4.200210
8,A,4.149655
9,D,4.125734


In [13]:
model.df_fi_final # финальный датафрейм важностей фичей  

,index,importance_benchmark,importance_permut,final_importance
0,F,230.534286,4.124747,226.409539
1,A,65.284569,4.149655,61.134913
2,J,63.469402,4.574215,58.895187
3,C,32.615238,4.260645,28.354593
4,G,29.989086,4.402431,25.586655
5,B,20.135345,4.357924,15.777422
6,D,8.951872,4.125734,4.826138
8,E,1.624735,4.506929,-2.882194
7,H,1.691387,4.580022,-2.888635
10,I,1.200047,4.200210,-3.000163


## Демонстрация метода на искусственных данных для задач классификации

In [14]:
# параметры для задачи классификации для разных эстиматоров
params_xgb = {'eta':0.03,
              'n_estimators':800,
              'subsample':0.9,
              'colsample_bytree':0.9,
              'max_depth':6,
              'objective':'binary:logistic',
              'use_label_encoder':False,
              'n_jobs':-1,
              'random_state':42}

params_ctb = {'depth':10,
              'subsample':0.33,
              'learning_rate':0.03,
              'loss_function':'Logloss',
              'random_state':42,
              'verbose':0}

params_lgb = {'learning_rate':0.03,
              'subsample':0.9,
              'colsample_bytree':0.9,
              'objective':'binary',
              'metric':'binary_logloss',
              'n_jobs':-1,
              'random_state':42,
              'verbose':-1,
              'force_col_wise':True}

params_dt = {'criterion':'gini',
             'max_depth':5,
             'random_state':42}

params_rf = {'criterion':'gini',
             'max_depth':5,
             'random_state':42,
             'n_estimators':50}

fit_params_xgb = {
    'early_stopping_rounds':100, 
    'eval_metric':['logloss', 'aucpr', 'auc'],
    'verbose':0}

fit_params_ctb = {
    'use_best_model':True,
    'early_stopping_rounds':200, 
    'verbose':0,
    'plot':False}

fit_params_lgb = {
    'early_stopping_rounds':100, 
    'eval_metric':['logloss', 'auc'],
    'verbose':0}

fit_params_rf = {}

fit_params_dt = {}


params_list = [params_xgb, 
               params_ctb, 
               params_lgb, 
               params_dt, 
               params_rf
              ]


fit_params_list = [fit_params_xgb, 
                   fit_params_ctb, 
                   fit_params_lgb, 
                   fit_params_dt, 
                   fit_params_rf
                  ]

models_list = ['xgboost', 
               'catboost', 
               'lightboost', 
               'decisiontree', 
               'randomforest'
              ] 

In [15]:
features_dict_target_perm = dict()
df_importance = dict()

for i in range(len(params_list)):
    estimator = utils_target_permutation.select_estimator(
        task_type='classification', 
        model_type=models_list[i], 
        main_fit_params=params_list[i])
    
    print('\n                                  Модель: ', models_list[i])
    model = TargetPermutationSelection(
        main_estimator=estimator, 
        main_fit_params=fit_params_list[i],
        main_prep_pipe=prep_pipe,
        main_features=num_columns + cat_columns,
        
        X_train=sample[num_columns + cat_columns],
        y_train=sample[target], 
        main_metric='gini',
        model_type=models_list[i])
    
    model.calculate_permutation()
    
    print('\n     Модель: ', models_list[i])
    features_dict_target_perm[models_list[i]] = model.select_features(select_type='tyta')
    df_importance[models_list[i]] = model.df_fi_final


                                  Модель:  xgboost
============================== 1. Обучение benchmark-модели! ==============================
BEST ITERATION:  55
gini на обучающей выборке: 99.489
gini на проверочной выборке: 99.162
============================== 2. Обучение target permutation-модели! ==============================
BEST ITERATION:  36
gini на обучающей выборке: 56.275
gini на проверочной выборке: 1.109

     Модель:  xgboost
10 признаков было до Target Permutation
6 признаков после Target Permutation

                                  Модель:  catboost
============================== 1. Обучение benchmark-модели! ==============================
BEST ITERATION:  495
gini на обучающей выборке: 99.999
gini на проверочной выборке: 99.025
============================== 2. Обучение target permutation-модели! ==============================
BEST ITERATION:  0
gini на обучающей выборке: 21.822
gini на проверочной выборке: 1.947

     Модель:  catboost
10 признаков было до Target

In [16]:
features_dict_target_perm

{'xgboost': ['F', 'J', 'A', 'G', 'C', 'B'],
 'catboost': ['F', 'B', 'G', 'J', 'D'],
 'lightboost': ['F', 'G', 'A', 'J', 'C', 'B'],
 'decisiontree': ['F', 'B', 'E'],
 'randomforest': ['F', 'A', 'J']}

In [17]:
print(utils_target_permutation.intersection_set(features_dict_target_perm, 3))

{'J', 'G', 'B', 'A', 'F'}


In [18]:
# отбор фичей при помощи взвешенной частоты фичей
utils_target_permutation.aggregated_feature_selection(
    feat_dist = features_dict_target_perm, 
    mean_score_select = True)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобраны фичи, которые имеют скор больше среднего.
Отобрано 5 фичей


,features,score
7,F,100.0000
1,J,63.3600
5,B,59.6196
6,A,50.4000
2,G,48.7800


In [19]:
# отбор фичей при помощи взвешенной частоты фичей
utils_target_permutation.aggregated_feature_selection(
    feat_dist = features_dict_target_perm, 
    mean_score_select = False)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобрано 8 фичей


,features,score
7,F,100.0000
1,J,63.3600
5,B,59.6196
6,A,50.4000
2,G,48.7800
0,C,26.2440
4,E,16.2000
3,D,13.1220


## Демонстрация метода на данных Титаник

In [20]:
sample = pd.read_csv('../data/train_titanic.csv')

print('Размерность данных:', sample.shape)
sample.head(2)

Размерность данных: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [21]:
num_columns = [
    'Age', 'Fare'
]

cat_columns = [
    'Pclass', 'Sex', 'Embarked', 'SibSp', 'Parch'
]

target = ['Survived']

prep_pipe = base_pipe(
    num_columns=num_columns, 
    cat_columns=cat_columns)

Определены количественные и категориальные переменные!


In [22]:
%%time
features_dict_target_perm = dict()
df_importance = dict()

for i in range(len(params_list)):
    estimator = utils_target_permutation.select_estimator(
        'classification', 
        models_list[i], 
        params_list[i])
    
    model = TargetPermutationSelection(
        main_estimator=estimator, 
        main_fit_params=fit_params_list[i],
        main_prep_pipe=prep_pipe,
        main_features=num_columns + cat_columns,
        
        X_train=sample[num_columns + cat_columns],
        y_train=sample[target], 
        
        main_metric='gini',
        model_type=models_list[i])
    
    model.calculate_permutation()
    
    print('Модель: ', models_list[i], '\n')
    features_dict_target_perm[models_list[i]] = model.select_features()
    df_importance[models_list[i]] = model.df_fi_final

============================== 1. Обучение benchmark-модели! ==============================
BEST ITERATION:  11
gini на обучающей выборке: 85.281
gini на проверочной выборке: 73.976
============================== 2. Обучение target permutation-модели! ==============================
BEST ITERATION:  4
gini на обучающей выборке: 59.954
gini на проверочной выборке: 9.411
Модель:  xgboost 

7 признаков было до Target Permutation
7 признаков после Target Permutation
============================== 1. Обучение benchmark-модели! ==============================
BEST ITERATION:  98
gini на обучающей выборке: 95.462
gini на проверочной выборке: 74.220
============================== 2. Обучение target permutation-модели! ==============================
BEST ITERATION:  24
gini на обучающей выборке: 87.064
gini на проверочной выборке: 2.393
Модель:  catboost 

7 признаков было до Target Permutation
7 признаков после Target Permutation
============================== 1. Обучение benchmark-модели! =====

In [23]:
features_dict_target_perm

{'xgboost': ['Sex', 'Pclass', 'Parch', 'SibSp', 'Embarked', 'Fare', 'Age'],
 'catboost': ['Sex', 'Pclass', 'SibSp', 'Age', 'Embarked', 'Parch', 'Fare'],
 'lightboost': ['Sex', 'Pclass', 'Age', 'Parch', 'Embarked', 'SibSp', 'Fare'],
 'decisiontree': ['Sex', 'Pclass', 'Parch', 'Embarked', 'SibSp'],
 'randomforest': ['Sex',
  'Pclass',
  'Fare',
  'Embarked',
  'Parch',
  'SibSp',
  'Age']}

In [24]:
print(utils_target_permutation.intersection_set(features_dict_target_perm, 3))

{'Sex', 'SibSp', 'Pclass', 'Age', 'Embarked', 'Parch', 'Fare'}


In [25]:
# отбор фичей при помощи взвешенной частоты фичей
utils_target_permutation.aggregated_feature_selection(
    feat_dist = features_dict_target_perm, 
    mean_score_select = True)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобраны фичи, которые имеют скор больше среднего.
Отобрано 3 фичей


,features,score
0,Sex,100.0000
2,Pclass,90.0000
6,Parch,71.9118


In [26]:
utils_target_permutation.aggregated_feature_selection(
    feat_dist = features_dict_target_perm, 
    mean_score_select = False)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобрано 7 фичей


,features,score
0,Sex,100.00000
2,Pclass,90.00000
6,Parch,71.91180
4,Embarked,68.52600
1,SibSp,67.52160
3,Age,52.03764
5,Fare,49.26744


# Задача Регрессии 

## Генерация искуственных данных

In [27]:
from sklearn.datasets import make_regression

X,y = make_regression(n_samples=10000,
                      n_features=10,
                      n_informative=5,
                      n_targets=1,
                      noise=25,
                      random_state=2022)

columns = ['A','B','C','D','E','F','G','H','I','J']
sample = pd.DataFrame(X, columns=columns)
sample['target'] = y
target = 'target'

In [28]:
num_columns = columns
cat_columns = []
target = ['target']

#pipline
prep_pipe = base_pipe(num_columns=num_columns,
                     cat_columns=cat_columns)

Определены только количественные переменные!


In [29]:
# Определение стандартных гиперпараметров моделей и список моделей
params_xgb = {
    'eta':0.01,
    'n_estimators':1000,
    'subsample':0.8,
    'colsample_bytree':0.8,
    'max_depth':6,
    'objective':'reg:squarederror',
    'n_jobs':-1,
    'random_state':42}

params_ctb = {
    'learning_rate':0.01,
    'iterations':1000,
    'subsample':0.8,
    'colsample_bylevel':0.8,
    'depth':6,
    'loss_function':'MAPE',
    'thread_count':-1,
    'random_state':42,
    'custom_metric':'MAE',
    'verbose':0}

params_lgb = {
    'learning_rate':0.01,
    'n_estimators':1000,
    'subsample':0.9,
    'colsample_bytree':0.6,
    'max_depth':6,
    'objective':'regression',
    'n_jobs':-1,
    'random_state':42}

params_dt = {'max_depth':5,
             'random_state':42}

params_rf = {'max_depth':5,
             'random_state':42,
             'n_estimators':50}

fit_params_xgb = {
    'early_stopping_rounds':200, 
    'eval_metric':'rmse',
    'verbose':0}

fit_params_ctb = {
    'use_best_model':True,
    'early_stopping_rounds':200, 
    'verbose':0,
    'plot':False}

fit_params_lgb = {
    'early_stopping_rounds':200, 
    'eval_metric':'rmse',
    'verbose':0}

fit_params_rf = {}

fit_params_dt = {}


params_list = [params_xgb, 
               params_ctb, 
               params_lgb, 
               params_dt, 
               params_rf
              ]

fit_params_list = [fit_params_xgb, 
                   fit_params_ctb, 
                   fit_params_lgb, 
                   fit_params_dt, 
                   fit_params_rf]

models_list = ['xgboost', 
               'catboost', 
               'lightboost', 
               'decisiontree', 
               'randomforest']

In [30]:
%%time
features_dict_target_perm = dict()
df_importance = dict()

for i in range(len(params_list)):
    estimator = utils_target_permutation.select_estimator('regression', models_list[i], params_list[i])
    
    model = TargetPermutationSelection(
        main_estimator=estimator, 
        main_fit_params=fit_params_list[i],
        main_prep_pipe=prep_pipe,
        main_features=num_columns + cat_columns,
        
        X_train=sample[num_columns + cat_columns],
        y_train=sample[target], 
        main_metric='mape',
        model_type=models_list[i])
    
    model.calculate_permutation()
    
    print('Модель: ', models_list[i], '\n')
    features_dict_target_perm[models_list[i]] = model.select_features()
    df_importance[models_list[i]] = model.df_fi_final

============================== 1. Обучение benchmark-модели! ==============================
BEST ITERATION:  999
mape на обучающей выборке: 0.849
mape на проверочной выборке: 1.157
============================== 2. Обучение target permutation-модели! ==============================
BEST ITERATION:  0
mape на обучающей выборке: 1.010
mape на проверочной выборке: 1.035
Модель:  xgboost 

10 признаков было до Target Permutation
10 признаков после Target Permutation
============================== 1. Обучение benchmark-модели! ==============================
BEST ITERATION:  996
mape на обучающей выборке: 0.758
mape на проверочной выборке: 0.913
============================== 2. Обучение target permutation-модели! ==============================
BEST ITERATION:  42
mape на обучающей выборке: 0.997
mape на проверочной выборке: 1.002
Модель:  catboost 

10 признаков было до Target Permutation
10 признаков после Target Permutation
============================== 1. Обучение benchmark-модели! =====

In [31]:
features_dict_target_perm

{'xgboost': ['G', 'E', 'J', 'A', 'B', 'C', 'D', 'F', 'I', 'H'],
 'catboost': ['G', 'E', 'J', 'A', 'I', 'H', 'B', 'F', 'D', 'C'],
 'lightboost': ['G', 'E', 'J', 'A', 'H', 'B', 'D'],
 'decisiontree': ['G', 'E', 'J'],
 'randomforest': ['G', 'E', 'J', 'F', 'A', 'I', 'D', 'H']}

In [32]:
print(utils_target_permutation.intersection_set(features_dict_target_perm , 3))

{'I', 'J', 'G', 'D', 'E', 'B', 'A', 'F', 'H'}


In [33]:
# отбор фичей при помощи взвешенной частоты фичей
utils_target_permutation.aggregated_feature_selection(
    feat_dist = features_dict_target_perm, 
    mean_score_select = True)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобраны фичи, которые имеют скор больше среднего.
Отобрано 4 фичей


,features,score
3,G,100.000
5,E,90.000
2,J,81.000
7,A,56.862


In [34]:
utils_target_permutation.aggregated_feature_selection(
    feat_dist = features_dict_target_perm, 
    mean_score_select = False)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобрано 10 фичей


,features,score
3,G,100.000000
5,E,90.000000
2,J,81.000000
7,A,56.862000
9,H,42.246148
4,D,40.495804
6,B,35.560620
8,F,33.711876
0,I,33.541144
1,C,19.558210


## Демонстрация метода на данных Стоимость жилья

In [35]:
sample = pd.read_csv('../data/train_houseprice.csv')

print('Размерность данных:', sample.shape)
sample.head(2)

Размерность данных: (1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [36]:
num_columns = [
    'LotArea', 'LotFrontage', 'TotalBsmtSF', 'PoolArea', 'FullBath'
]

cat_columns = [
    'BldgType', 'CentralAir', 'GarageQual', 'Fence', 'SaleType'
]

target = ['SalePrice']

prep_pipe = base_pipe(
    num_columns=num_columns, 
    cat_columns=cat_columns)

Определены количественные и категориальные переменные!


In [37]:
%%time
features_dict_target_perm = dict()
df_importance = dict()

for i in range(len(params_list)):
    estimator = utils_target_permutation.select_estimator('regression', models_list[i], params_list[i])
    
    model = TargetPermutationSelection(
        main_estimator=estimator, 
        main_fit_params=fit_params_list[i],
        main_prep_pipe=prep_pipe,
        main_features=num_columns + cat_columns,
        
        X_train=sample[num_columns + cat_columns],
        y_train=sample[target], 
        main_metric='mape',
        model_type=models_list[i])
    
    model.calculate_permutation()
    
    print('Модель: ', models_list[i], '\n')
    features_dict_target_perm[models_list[i]] = model.select_features()
    df_importance[models_list[i]] = model.df_fi_final

============================== 1. Обучение benchmark-модели! ==============================
BEST ITERATION:  795
mape на обучающей выборке: 0.087
mape на проверочной выборке: 0.158
============================== 2. Обучение target permutation-модели! ==============================
BEST ITERATION:  447
mape на обучающей выборке: 0.266
mape на проверочной выборке: 0.350
Модель:  xgboost 

10 признаков было до Target Permutation
10 признаков после Target Permutation
============================== 1. Обучение benchmark-модели! ==============================
BEST ITERATION:  999
mape на обучающей выборке: 0.225
mape на проверочной выборке: 0.239
============================== 2. Обучение target permutation-модели! ==============================
BEST ITERATION:  0
mape на обучающей выборке: 0.285
mape на проверочной выборке: 0.314
Модель:  catboost 

10 признаков было до Target Permutation
10 признаков после Target Permutation
============================== 1. Обучение benchmark-модели! ====

In [38]:
features_dict_target_perm

{'xgboost': ['FullBath',
  'TotalBsmtSF',
  'BldgType',
  'PoolArea',
  'SaleType',
  'GarageQual',
  'CentralAir',
  'LotArea',
  'Fence',
  'LotFrontage'],
 'catboost': ['TotalBsmtSF',
  'FullBath',
  'GarageQual',
  'SaleType',
  'LotFrontage',
  'BldgType',
  'PoolArea',
  'Fence',
  'LotArea',
  'CentralAir'],
 'lightboost': ['TotalBsmtSF',
  'FullBath',
  'LotArea',
  'LotFrontage',
  'SaleType',
  'BldgType'],
 'decisiontree': ['TotalBsmtSF',
  'FullBath',
  'BldgType',
  'SaleType',
  'PoolArea'],
 'randomforest': ['TotalBsmtSF',
  'FullBath',
  'BldgType',
  'SaleType',
  'PoolArea',
  'Fence',
  'GarageQual',
  'CentralAir',
  'LotFrontage']}

In [39]:
print(utils_target_permutation.intersection_set(features_dict_target_perm))

{'CentralAir', 'BldgType', 'LotArea', 'FullBath', 'PoolArea', 'GarageQual', 'SaleType', 'LotFrontage', 'Fence', 'TotalBsmtSF'}


In [40]:
# отбор фичей при помощи взвешенной частоты фичей
utils_target_permutation.aggregated_feature_selection(
    feat_dist = features_dict_target_perm, 
    mean_score_select = True)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобраны фичи, которые имеют скор больше среднего.
Отобрано 4 фичей


,features,score
9,TotalBsmtSF,98.0000
3,FullBath,92.0000
1,BldgType,72.2196
6,SaleType,69.9840


In [41]:
utils_target_permutation.aggregated_feature_selection(
    feat_dist = features_dict_target_perm, 
    mean_score_select = False)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобрано 10 фичей


,features,score
9,TotalBsmtSF,98.000000
3,FullBath,92.000000
1,BldgType,72.219600
6,SaleType,69.984000
4,PoolArea,51.452820
7,LotFrontage,44.059754
5,GarageQual,38.638620
2,LotArea,34.375282
8,Fence,29.985082
0,CentralAir,27.943168
